In [6]:
import warnings
import anndata
import scanpy as sc
import Cell_BLAST as cb
import os
import numpy as np
import pandas as pd

warnings.simplefilter("ignore")
cb.config.RANDOM_SEED = 0


In [7]:
adata = sc.read('/data/share/cnic02/data/cpa_binaries/datasets/GSM_new.h5ad') # 14811*4999
# data = sc.read_csv('/data/share/scRNA_10X_data/pipeline_by_ourteam/NO1_data/SRR9621088_count.csv')
model_save_dir = './pretrained_models/'

In [8]:
adata.obs

,sample,size_factor,n.umi,hash_umis,pval,qval,top_to_second_best_ratio,top_oligo,drug,dose,...,mt_frac,product_dose,louvain,dose_val,cell_type,drug_dose_name,cov_drug_dose_name,condition,control,split
index,,,,,,,,,,,,,,,,,,,,,
A01_A01_RT_390,sciChem18,0.849329,3430.0,16.0,1.404159e-140,3.650813e-139,15.005464,Nutlin_0.5_AB02,Nutlin,0.5,...,0.089213,Nutlin 0.5,1,0.005,A549,Nutlin_0.005,A549_Nutlin_0.005,Nutlin,0,test
A01_A01_RT_396,sciChem18,1.612486,6512.0,35.0,0.000000e+00,0.000000e+00,2.379168,Dex_50_BC10,Dex,50.0,...,0.045455,Dex 50.0,7,0.500,A549,Dex_0.5,A549_Dex_0.5,Dex,0,ood
A01_A01_RT_400,sciChem18,0.786681,3177.0,23.0,3.035262e-222,9.409311e-221,20.920000,Nutlin_0_AB04,Vehicle,0.0,...,0.094429,Vehicle 0.0,3,1.000,A549,Vehicle_1.0,A549_Vehicle_1.0,Vehicle,1,train
A01_A01_RT_401,sciChem18,1.258641,5083.0,69.0,0.000000e+00,0.000000e+00,1.892483,BMS_0.1_BH11,BMS,0.1,...,0.084596,BMS 0.1,1,0.001,A549,BMS_0.001,A549_BMS_0.001,BMS,0,train
A01_A01_RT_402,sciChem18,3.432476,13862.0,104.0,0.000000e+00,0.000000e+00,1.262781,Nutlin_10_AA03,Nutlin,10.0,...,0.063771,Nutlin 10.0,3,0.100,A549,Nutlin_0.1,A549_Nutlin_0.1,Nutlin,0,test
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
H12_B02_RT_745,sciChem18,0.731463,2954.0,63.0,0.000000e+00,0.000000e+00,63.363772,Nutlin_1_AB01,Nutlin,1.0,...,0.072444,Nutlin 1.0,3,0.010,A549,Nutlin_0.01,A549_Nutlin_0.01,Nutlin,0,train
H12_B02_RT_750,sciChem18,0.611616,2470.0,116.0,0.000000e+00,0.000000e+00,143.054022,Nutlin_50_AA10,Nutlin,50.0,...,0.014575,Nutlin 50.0,5,0.500,A549,Nutlin_0.5,A549_Nutlin_0.5,Nutlin,0,ood
H12_B02_RT_752,sciChem18,3.208135,12956.0,63.0,0.000000e+00,0.000000e+00,4.731265,BMS_0.1_BG07,BMS,0.1,...,0.088916,BMS 0.1,3,0.001,A549,BMS_0.001,A549_BMS_0.001,BMS,0,train


In [5]:
adata.to_df()

index,SCYL3,C1orf112,GCLC,STPG1,KRIT1,RAD52,MYH16,BAD,KLHL13,ICA1,...,AL589669.1,AL391117.1,AC092718.8,AC015802.6,AC087565.3,AC006511.5,AC079781.5,AL034417.3,AC087269.2,AL358472.5
index,,,,,,,,,,,,,,,,,,,,,
A01_A01_RT_390,0.000000,0.000000,0.778132,0.00000,0.000000,0.000000,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000
A01_A01_RT_396,0.000000,0.000000,1.247217,0.00000,0.482525,0.000000,0.0,0.0,0.0,0.482525,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000
A01_A01_RT_400,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000
A01_A01_RT_401,0.000000,0.000000,1.752160,0.00000,0.584731,0.000000,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000
A01_A01_RT_402,0.000000,0.000000,1.203176,0.00000,0.000000,0.000000,0.0,0.0,0.0,0.255676,...,0.0,0.0,0.0,0.0,0.0,0.0,0.255676,0.0,0.0,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
H12_B02_RT_745,0.000000,0.000000,0.861676,0.00000,0.861676,0.000000,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000
H12_B02_RT_750,0.000000,0.000000,0.000000,0.00000,0.000000,0.968888,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000
H12_B02_RT_752,0.484532,0.000000,1.157495,0.00000,0.271330,0.271330,0.0,0.0,0.0,0.939436,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000


In [9]:
adata.var

,id,n_cells,highly_variable,means,dispersions,dispersions_norm
index,,,,,,
SCYL3,ENSG00000000457.13,1582,True,0.087527,0.568619,0.608302
C1orf112,ENSG00000000460.16,2672,True,0.152932,0.619716,0.820082
GCLC,ENSG00000001084.10,11857,True,1.018276,0.905976,0.571599
STPG1,ENSG00000001460.17,1147,True,0.062389,0.567663,0.604339
KRIT1,ENSG00000001631.15,6321,True,0.388936,0.631260,0.501368
...,...,...,...,...,...,...
AC006511.5,ENSG00000284697.1,304,True,0.016723,0.587521,0.686645
AC079781.5,ENSG00000284707.1,1223,True,0.067030,0.611512,0.786081
AL034417.3,ENSG00000284716.1,202,True,0.011656,0.569787,0.613141


In [ ]:
axes = cb.data.find_variable_genes(adata)
cb_model = cb.directi.fit_DIRECTi(
    adata, genes=adata.var.query("variable_genes").index,
    latent_dim=10, cat_dim=20
)
# model.save("/data/share/cnic02/models/cb_models/gem_new_100")
# del model